<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/Scraping/main_rankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests

def get_ranking_from_call(url_business, lang, sort, query):
    headers = [{"name": "Accept", "value": "*/*"}, {"name": "Accept-Encoding", "value": "gzip, deflate, br"},
               {"name": "Accept-Language", "value": "it-IT,it;q=0.8,en-US;q=0.5,en;q=0.3"},
               {"name": "Connection", "value": "keep-alive"},
               {"name": "Content-Type", "value": "application/x-www-form-urlencoded; charset=utf-8"}, {"name": "Cookie",
                                                                                                       "value": "qntcst=D; hl=en_US; wdi=1|3C26116D69138F61|0x1.78d019f71a444p+30|a7756ff94751d3a9; _ga=GA1.2.3C26116D69138F61; location=%7B%22city%22%3A+%22New+York%22%2C+%22state%22%3A+%22NY%22%2C+%22country%22%3A+%22US%22%2C+%22latitude%22%3A+40.713%2C+%22longitude%22%3A+-74.0072%2C+%22max_latitude%22%3A+40.8523%2C+%22min_latitude%22%3A+40.5597%2C+%22max_longitude%22%3A+-73.7938%2C+%22min_longitude%22%3A+-74.1948%2C+%22zip%22%3A+%22%22%2C+%22address1%22%3A+%22%22%2C+%22address2%22%3A+%22%22%2C+%22address3%22%3A+null%2C+%22neighborhood%22%3A+null%2C+%22borough%22%3A+null%2C+%22provenance%22%3A+%22YELP_GEOCODING_ENGINE%22%2C+%22display%22%3A+%22New+York%2C+NY%22%2C+%22unformatted%22%3A+%22New+York%2C+NY%2C+US%22%2C+%22accuracy%22%3A+4.0%2C+%22language%22%3A+null%7D; xcj=1|Ptt9P03gfc75x_PBT9zmqCkUuSuyB7PR-wWUBvABNi4; __qca=P0-60561249-1581956668708; G_ENABLED_IDPS=google; __cfduid=db8764ff59d8028a6c2e1b214867927d81583160194; _gid=GA1.2.2014867238.1583835527; bse=05dcd9d5de304ef0b1d9a76fa768b10f; sc=8a1ca0dbc2; pid=505721aa4569e7bb"},
               {"name": "Host", "value": "www.yelp.com"},
               {"name": "Referer", "value": "https://www.yelp.com/biz/noche-de-margaritas-new-york"},
               {"name": "TE", "value": "Trailers"}, {"name": "User-Agent",
                                                     "value": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0"},
               {"name": "X-Requested-By-React", "value": "true"},
               {"name": "X-Requested-With", "value": "XMLHttpRequest"}]
    headers_ok = {}
    for header in headers:
        temp = {
            header['name']: header['value']
        }
        headers_ok.update(temp)

    x = 0
    reviews_list = []
    position = 1
    url = url_business + "/review_feed?rl=" + lang + "&sort_by=" + sort + "&q=" + query

    while 1:
        if x == 0:
            page_load = requests.get(url + '&start=', headers=headers_ok)
        else:
            page_load = requests.get(url + '&start=' + str(x), headers=headers_ok)
        x = x + 20
        reviews = page_load.json()['reviews']
        # print(json.dumps(reviews, indent=4, sort_keys=True))
        if not reviews:
            break
        for review in reviews:

            # Qui potete modificare, prendendovi tutti i dati che volete e scegliete anche in che tipo di struttura dati
            # metterli. Io ho usato una semplice lista, ma se dovete prendere tanti dati di tante review vi consiglio un
            # dataframe
            #reviews_list.append((position, review['userId'], review['user']['reviewCount']))
            reviews_list.append(review['userId'])

            position = position + 1
    print(reviews_list)
    return reviews_list

In [0]:
def RBO(l1, l2, p):
    """
        Calculates Ranked Biased Overlap (RBO) score. 
        l1 -- Ranked List 1
        l2 -- Ranked List 2
    """
    if l1 == None: l1 = []
    if l2 == None: l2 = []
    
    sl,ll = sorted([(len(l1), l1),(len(l2),l2)])
    s, S = sl # s = length of smaller list, S = Smaller List
    l, L = ll # l = length of longer list, L = Longer list
    if s == 0: return 0

    # Calculate the overlaps at ranks 1 through l 
    # (the longer of the two lists)
    ss = set([]) # contains elements from the smaller list till depth i
    ls = set([]) # contains elements from the longer list till depth i
    x_d = {0: 0} # overlap holds number of common elements at depth d
    sum1 = 0.0
    for i in range(l):
        x = L[i]
        y = S[i] if i < s else None
        d = i + 1
        
        # if two elements are same then 
        # we don't need to add to either of the set
        if x == y: 
            x_d[d] = x_d[d-1] + 1.0
        # else add items to respective list
        # and calculate overlap
        else: 
            ls.add(x) 
            if y != None: ss.add(y)
            x_d[d] = x_d[d-1] + (1.0 if x in ss else 0.0) + (1.0 if y in ls else 0.0)     
        #calculate average overlap
        sum1 += x_d[d]/d * pow(p, d)
        
    sum2 = 0.0
    for i in range(l-s):
        d = s+i+1
        sum2 += x_d[d]*(d-s)/(d*s)*pow(p,d)

    sum3 = ((x_d[l]-x_d[s])/l+x_d[s]/s)*pow(p,l)

    # Equation 32
    rbo_ext = (1-p)/p*(sum1+sum2)+sum3
    return rbo_ext

In [0]:
# ESEMPIO DI CHIAMATA AL METODO
def compare_ranking_restaurant(business_id):
  #business_id = 'ci0-GO_iMb0NdTCUYWjocQ'
  language = 'en'

  sorting = 'relevance_desc'  # relevance è l'ordinamento di default di Yelp, descending
                              # altri valori possibili: date_desc, date_asc, relevance_asc
  query = ''
  ranking = get_ranking_from_call("https://www.yelp.com/biz/" + business_id, language, sorting, query)

  sorting = 'date_desc'  # relevance è l'ordinamento di default di Yelp, descending
                        # altri valori possibili: date_desc, date_asc, relevance_asc
  ranking2 = get_ranking_from_call("https://www.yelp.com/biz/" + business_id, language, sorting, query)

  return RBO(ranking,ranking2,p = 0.93)

In [6]:
import csv
#creo e apro un file .csv per scrivere
restaurant_RBO = open('Restaurant_RBO.csv', 'w')

#creo il writer
writer = csv.writer(restaurant_RBO)

#creo header del csv
header = ['business_id', 'business_name', 'rbo']
writer.writerow(header)

writer.writerow(['ci0-GO_iMb0NdTCUYWjocQ',
                 'Noche De Margaritas',
                 compare_ranking_restaurant('ci0-GO_iMb0NdTCUYWjocQ')])
#salvo e chiudo il file .csv
restaurant_RBO.close()

['RJH925_Ci598EuRx9JoGwA', 'cd2eHx12PMHIws65ChDfYQ', 'nRV0VIm-7fwB-_UgL5PQjA', 'SziKmSU5nW1TYnQE1s-zGA', 'xpDVPSLlZnU8MQaOS2HAwA', 'E6-NNryKZzt0eBlQFvqtYw', 'ouODopBKF3AqfCkuQEnrDg', 'uDBnapEGywTQ0ovaCu7Orw', 'rnvljaN99l4ZcjH69rKLzQ', 'KRs1_BWofrey9L2jL5xeeA', 'OpkcCxKVQEi1eTtpCho87Q', 'WwZrmap7gLB5Bm_5l-L9LA', 'xtVcWcIKVORcBKhmdbrFiA', 'Qy0zhduJMSLeOaqDaXYGJQ', 'BSat-Dsl5CDCgcX6INo83w', 'Z-w8e86U76_-UHmnR_on2g', 'bV9IYVYOkbm8gEyBd8Gq5g', 'sMfzdYpg2oUYUUd-9EnnIQ', 'OEdWrFmr01T9pfDb6Auu4g', 'hxknn8DNhg7XDFCFQFuzpQ', 'aVRPfNY7aaR3DZtAkQY_Hw', 't0Y_Hgg7H8W030BNcQvl1A', 'rahkAG8IZ9sqZPA2gPRDwg', 'uuJ-QTsCXe69ABdxOWKNOg', 'LiA3AUNmTH9ALw6PWR45JA', 'HwIIv3dU2bjh36XXUecxog', 'Xqo1ru1F7srvbUJaCNHkvQ', 'Zq6uLNtQ_LSbiDk1BNewQg', 'nxO6hC_qtFkKYe07I5bPAQ', 'qRB3eBE7dm1uEe6VBxxC1Q', '7iruCfIKVc7sIIW84vbgMw', '4E8xk4Q4npb5QhvVu7jQ1A', 'BO5uPZqVspbOLsDLE5c5jQ', '4E6WGB8XZrSa_26UcVTSvQ', 'whp2b_6melUtfkTMr0Rs3A', 'qsZ6hRVKb7HaWu2lOz4YEw', 'NLnxvTdm5MC8JBzlRZYtug', 'qEhqu0XB-ius765_EsVWFw', 'h2w160yqcE